In [189]:
import pandas as pd 
import numpy as np
import re
import time
from pandas import DataFrame, Series
from pprint import pprint
from konlpy.tag import Mecab

In [190]:
import matplotlib.pyplot as plt
import matplotlib as mpl
# chart size setting
plt.rc('figure', figsize=(10, 6)) 
# DF, Series row count
pd.options.display.max_rows = 20 
mpl.rc('font', family='New Gulim')

import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
import platform
path = "c:/Windows/Fonts/malgun.ttf" 
from matplotlib import font_manager, rc 
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic') 
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')

In [191]:
df = pd.read_csv('./question_tag.csv', index_col=0, encoding='utf-8-sig')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34911 entries, 0 to 34910
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  34911 non-null  object
 1   main      34911 non-null  object
 2   meaning   34911 non-null  object
dtypes: object(3)
memory usage: 1.1+ MB


In [192]:
df[df['main'].str.contains('요청')][150:200]

,sentence,main,meaning
3880,네 피클1개와 콜라 추가해 주세요,메뉴추가문의요청,문의
3882,너무 오래걸리는데 혹시 방문 포장 가능하나요?,방문포장문의요청,문의
3890,그럼 스파게티 구폰 1장이랑 토마토 스파게티 2개를 더 주문할께요,메뉴추가문의요청,문의
3905,무슨 치킨이 가장 맛있나요?,베스트메뉴문의추천요청,추천
3916,콜라는 사이즈업 해주세요,사이즈-업문의요청,문의
...,...,...,...
4658,팔찌 제품 따로 있나요?,종류별액세서리제품문의요청,문의
4659,고객님이 하실 거예요?,종류별액세서리제품문의요청,문의
4663,이거 샘플이 14케이인데 뭘로 봐드릴까요?,종류별액세서리제품문의요청,문의
4668,굵은 게 이거 다예요?,종류별액세서리제품문의요청,문의


In [193]:
#meaning column 에 한글이 있음 나중에 vect 연산 처리할 때 문제 있을 수 있음
df.loc[df[df.meaning =='문의'].index, 'meaning'] = 'Q'
df.loc[df[df.meaning =='추천'].index, 'meaning'] = 'R'

In [194]:
df.head()

,sentence,main,meaning
0,지금 배달되나요?,배달가능문의,Q
1,짬뽕류는 어떤 게 있나요? 잘 나가는 짬뽕 있나요?,베스트메뉴문의추천요청,R
2,전복 들어가는 거는 특해물 짬뽕 시켜야 돼요?,메뉴정보문의,Q
3,전복 들어가고 여러 가지 또 딴 것도 들어가죠?,메뉴정보문의,Q
4,마차이 짬뽕밥은 돼지고기 들어가나요?,메뉴정보문의,Q


In [195]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['sentence'].values
                                                    , df['meaning'].values
                                                   ,test_size=0.25) #디폴트 값
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((26183,), (8728,), (26183,), (8728,))

In [196]:
korean_re = re.compile('[^ ㄱ-ㅣ가-힣]+')

In [197]:
s_file_name = open('./한국어 불용어(stop_words)정리.txt','r', encoding='utf-8')
stop_words_list = []
for line in s_file_name.readlines():
    stop_words_list.append(line[:-1])

In [198]:
#불용어 처리 및 추출된 형태소만 출력되는 함수
def text_to_morphs(text):
    text = korean_re.sub('',text)
    mecab = Mecab()
    text = ' '.join(token for token in mecab.morphs(text) if token not in stop_words_list)
    return text

In [199]:
text = '불용어 처리를 위한 테스트용 텍스트 문장입니다. 민호님 부탁해요!'
text_to_morphs(text)

'불 용어 처리 위한 테스트 용 텍스트 문장 입니다 민호 님 부탁'

In [200]:
#위 x_train , test 데이터의  sentence 부분을 한글, 공백만 남긴 후 빈 리스트에 저장
X_train = [text_to_morphs(x) for x in X_train]
X_test = [text_to_morphs(x) for x in X_test]

In [201]:
X_train[:3]

['빨리 나요 밥', '날개 봉 그건', '양 갱이 어른 좋']

In [202]:
X_test[:3]

['다 세트 해야 되 나요', '레벨 마다 교재 새로 사', '게 한테 더 잘 어울리 나요']

In [203]:
pprint(X_train) #x에는 질문이

['빨리 나요 밥',
 '날개 봉 그건',
 '양 갱이 어른 좋',
 '상품권 신세계 상품권 만 되 나요',
 '앞 쓰여 있 던데 없',
 '애 이랑 따로 있 어야 조용 할 꺼 은데 분리 곳 조용 곳 있 나요',
 '빨리 게 김밥 죠',
 '아직 세트 할인 이벤트 있 나요',
 '요런 옷 예요',
 '이거 마음 드 는데 번 메 봐도 될까요',
 '요 카키색 코트 예요',
 '선행 하나 요',
 '앞 신용카드 사절 라고 되 있 데 체크카드 되 나요',
 '교재비 요',
 '색상 좋 으세요',
 '사장 님 수육 거리 뭐 있',
 '내과 서 피 검사 보 셨',
 '커플 석 거 있 나요',
 '이거 말 평소 잘 할 있 거 없',
 '할인 행사 한다고 었 는데 맞 나요',
 '교재비 매달 나요',
 '현재 흰색 검정 색 있 데 신 보 시 겠',
 '예요',
 '옷 예요',
 '맛있 사과 있 나요',
 '일 정도 걸리 나요',
 '색상 색상 밖에 없',
 '구 치 려고 는데 대 다이 있',
 '약밥 말 세요',
 '메뉴 있 거 만 있 라면 없',
 '유료 주차장 요',
 '세트 메뉴 면 가격 많이 싸 나요',
 '진로 면 고등학교 시작 경우 있 나요',
 '파마 걸리 나요',
 '요즘 어르신 생신 선물 이나 제일 잘 나가 케이크 거 에요',
 '클래식 곡 치 려면 걸릴까요',
 '주문 시 면 십 분 십 분 정도 걸리 세요',
 '시 영업 나요',
 '제품 어때요',
 '입 되 나요',
 '스타일 선호 아세요',
 '하루 숙박 비용 얼만 가요',
 '남녀 공용 인가 보',
 '회원 가격 인가요',
 '신어 볼 있 나요',
 '아플 마다 먹 거 예요',
 '남자 전용 미용실 맞 나요',
 '반코트 정도 할까요',
 '있 반찬 하나 씩 하나 요',
 '너무 요란 무늬 말 얌전 거 없',
 '선 게 잘 나가 요',
 '호텔 식당 일 층 아닌가요',
 '아이 결석 면 따로 연락 주 시 나요',
 '걸리 나요',
 '우유 있',
 '예약 가능 한가요',
 '샐러드 드레싱 있 나요'

 '본점 지점 다 있 죠',
 '크로스 백 없 나요',
 '주차 대 가능 한가요',
 '일 주일 번 면 돼요',
 '라인 면 미백 되 거 고요',
 '저번 탈색 번 했 는데 번 가능',
 '건성 인데 거 써야 나요',
 '우유 거 서 배달 가능 한가요',
 '배달 되 나요',
 '페이 결제 가능 한가요',
 '클리닉 추가 면 인가요',
 '배달 가능 한가요',
 '옆 테이블 이랑 붙여서 앉 아도 되 나요',
 '후불제 이용 가능 한가요',
 '반코트 자연 스럽 게 수선 되 가요',
 '칼라 장 뽑 으려고 는데요',
 '아저씨 귤 이거 꺼 예요',
 '겨울 용품 세일',
 '주 주말 캠 핑장 예약 싶 은데 되 나요',
 '짜 먹 거 정도 하나 요',
 '찜 닭 게 잘 나가 요',
 '사이즈 삼 십 팔 입 만 옷 뭐 있',
 '떡 팩 원 에요',
 '단화 종류 보여 주 세요',
 '잘 어울리 봐 주 세요',
 '혼자 먹 있 메뉴 있',
 '커피 안 들어가 죠',
 '목걸이 잘 나가 죠',
 '이게 잘 나가 거 죠',
 '오늘 날짜 예약 했 는데 확인 되 나요',
 '비타민 요',
 '들어가',
 '면접 있 어서 깔끔 게 자르 려고요',
 '가죽 예요',
 '면 셔츠 어떠신가요',
 '알약 못 먹 어서 가루약 제조 줄 있 나요',
 '미니 백 보여 주 실 있 나요',
 '빵 서울 내려오 거 예요',
 '샴푸 예요',
 '새로 개설 수능 반 교실 있',
 '단체 할인 가능 한가요',
 '귀걸이 파',
 '요 신발 요 사장 님',
 '에코 백 있 나요',
 '가격 싸',
 '일요일 안 세요',
 '애 데리 번 와볼',
 '소고기 불고기 시세 다 오른 거 에요 내린 거 에요',
 '날 신가요',
 '가방 가죽',
 '거 위주 나요',
 '가격 그거 이거 인데',
 '쌀 갖 면 하루 걸려요 이틀 걸려요',
 '시험 기간 만 이용 려는데 가능 한가요',
 '손 님 그러 신가요',
 '학년 있 나요',
 '세트 메뉴 말 단 품 추천 주 세요',
 '조식 포함 되 가격 인가요'

 '현금 영수증 발급 주',
 '돼지 앞 다 릿 살 좋 은데 드릴까요',
 '냉동 딸기 있 나요',
 '갈비 저건 먹',
 '잡채 파나 요',
 '일반 커피 비엔나 커피 랑 뭐 달라요',
 '가격 나요',
 '멸치 젓 넣 안 넣 못 나요',
 '체크인 할 있 나요',
 '방안 냉장고 음료 무료 입니까',
 '긴 머리 인데 파마 는데 걸려요',
 '위치 십니까',
 '입시 반 시간표 짜여져 있',
 '잼 들어간다 고요',
 '시 나요',
 '광범위 뒤 메 거 찾 으시',
 '떡 아침 나올까요',
 '가게 이름 저게 뜻 거 예요',
 '거기 저녁 시 세요',
 '귀걸이 구매 시 할인 혜택 있 나요',
 '성인 명과 어린이 들어갈 있 방 있 나요',
 '사이즈 잘 맞 는데 색 더 잘 어울리 나요',
 '커플 석 있 나요',
 '요거 뭐 예요',
 '장사 시 나요',
 '평상시 말 물놀 서 바를 거 예요',
 '회원 요금 에요',
 '더치커피 파나 요',
 '주문 해도 될까요',
 '여자 선생 님 없 으신 가요',
 '개월 이용 려는데 달 씩 카드 결제 되 나요',
 '컬러 두 가지 뿐 인가요',
 '서 만드 시',
 '애 거 하나 사 달 라고 가지 거 제일 많이 나가 요',
 '알 러 없 나요',
 '주차 할 있 공간 있',
 '주로 많이 나가 약 거 에요',
 '앞 보라색 털 달린 슬리퍼 씩',
 '이번 새로 나온 거 예요',
 '청소 매일 주 나요',
 '가격 원 적혀 있 데 맞 아요',
 '완성 반지 찾 있',
 '시시 티비 설치 되 있 나요',
 '요번 새로 나왔 다는 메뉴 뭐 예요',
 '요새 귀걸이 긴 거 많이 하나',
 '인가요',
 '딸기 요',
 '거 뭐 라 그래요',
 '체크 시 체크아웃 시 인가요',
 '불고기 버거 파나 요',
 '이거 가격 요',
 '방 개 가격 인가요',
 '금액 추가 할 테 니 세트 두 개 따로 포장 주 실 있 으세요',
 '디자인 마음 드 는데 작 아요',
 '펜 전체 테두리 아닌가요',
 '김밥 먹 으려고 는데 뭐 뭐 

 '월요일 시 죠',
 '카카 오페 이나 삼성 페이도 가능 할까요',
 '양념 이거 깨 속 있 건 뭐 요 초록색',
 '결재 할 카카오 페이 가능 한가요',
 '현금 면 할인 안 되 나요',
 '쿠폰 개 모아야 음료 무료 요',
 '정도 걸리 세 마리 려면 요',
 '천 원 안 돼요',
 '와이셔츠 사 려고 는데 있 나요',
 '가격 적힌 거 뭐 예요',
 '이용 대금 예요',
 '시 면 수업 다 끝나 요',
 '밥솥 있 는데 사용 되 나요',
 '밥 없이 전골 만',
 '곰피 팔 아요',
 '사과 인가요',
 '이거 세일 가격 에요',
 '금 연실 주 세요',
 '콜라 작 거 요',
 '배달 시 돼요',
 '말린 과일 다 원 씩 에요',
 '여자 커트',
 '학교 인가요',
 '이거 가격 예요',
 '일 주일 며칠 요',
 '카페 이름 분 에요',
 '반지 스타일 원 세요',
 '가지 러 와야 돼요',
 '약과 예요',
 '이불 세탁 해줘요',
 '태반 먹 으면 좋 아요',
 '잘 안 풀리 파마 추천 주 세요',
 '양파 들어왔 나요',
 '전화 주문 가능 한가요',
 '텀블러 담 으려면 사이즈 거 해야 돼요',
 '예약 문의 려는 데 요',
 '전날 전화 드려야 죠',
 '이건 디자인 굉장히 화려 한데요',
 '처방전 없 약 추가 구매 가능 죠',
 '와이파이 돼요',
 '가격',
 '커피숍 이나 건 전혀 없',
 '떡볶이 인분 포장 돼요',
 '떨어지 않 아요',
 '이건 코르덴 나왔 거 겨울 많이 었 겠 다 그죠',
 '채소 먹 어야 죠',
 '컬러 안 괜찮 아요',
 '다 세일 고요',
 '수학 기본 되 는데 심화 어려워 요',
 '이게 가죽 에요',
 '쉬 날 있',
 '화요일 맞 으시 죠',
 '세트 메뉴 번 보 시 겠',
 '요 회색 자켓 예요',
 '뚱 카 롱 있 나요',
 '가래떡 판 인가요',
 '메뉴 종류 찾 으시 나요',
 '이게 수제 화 예요',
 '가방 만 팔 천 원 에요',
 '이용 실 예정 신가요',
 '액세서리 있 나요',
 '

 '팔찌 예쁜데 가격 비슷 할까요',
 '',
 '보 신 보여 주 실 있 나요',
 '휴일 미리 공지 시 죠',
 '구 입니까',
 '요 가방 예요',
 '영어 말 기 대회 나가 는데 특별 상담 받 싶 은데요',
 '사이즈 신 으세요',
 '액체 되 있',
 '박 면 할인 없 나요',
 '클렌징 폼 하나 각질 제거 하나 카드 가능 한가요',
 '이건 비닐 에요',
 '마스크 있',
 '관광지 도보 갈 없 죠',
 '제품 똑같 바지 없 나요',
 '성인 학생 다 구별 되 있 나요',
 '배송 가능 지역 동 인가요',
 '바지 허리 만 줄이 시 나요',
 '피아노 전공 려는 학생 있 나요',
 '빵 주로 나와요',
 '오래 기다려야 될까요',
 '룸 배달 음식 시켜 먹 있 나요',
 '헤나 염색',
 '삼성 페이 결제 가능 한가요',
 '삼겹살 줄까 요',
 '당구장 짜장면 있 나요',
 '세일 품목 위 있 거 예요',
 '용각산 쿨 말씀 시 거 예요',
 '김밥 게 맛있',
 '사이즈 입 으세요',
 '등뼈 씩',
 '블라우스 없 나요',
 '예약 실 날짜 말씀 주 세요',
 '가입 안 그냥 후불 카드 해도 되 죠',
 '없',
 '이월 상품 보여 주 시 겠',
 '만 마시 되 죠',
 '꽃무늬 원피스 벌써 나왔',
 '귤 박스 인가요',
 '카페 라테 잔 샷 번 들어가 요',
 '시 시 영업',
 '맛있 과일 하나 만 추천 주',
 '매운 김밥 건 없 보',
 '체크 시 인가요',
 '가래떡 되 쌀 쓰 면서 만 원 인가요',
 '제작 드리 면 될까요',
 '상품 이번 디엠 행사 할인 구입 가능 신데요',
 '좋 염색 거 있',
 '방 침대 다 있 나요',
 '수업 에요',
 '현금 드리 면 할 거 없 죠',
 '젊 애 헤어 오일 많이 써요',
 '잘 탈 런가',
 '사과 되 나요',
 '세일 안',
 '회원 할인 되 나요',
 '간 부담 덜 나요',
 '예약 문 면 되 나요',
 '당도 가요',
 '그냥 염색 만 세요',
 '버스 내릴 건데 멀리 있 나요',


 '근처 와플 커피숍 말 커피숍 없 나요',
 '서 가지 러 면 요',
 '주차장 어딨',
 '애 그냥 액세서리 거 예요',
 '게장 간장 게장 다 가요',
 '현금 없 어서 입금 드려도 될까요',
 '이게 겨울옷 인가요',
 '일요일 마다 쉬 나요',
 '배달 비 따로 받 으시',
 '신상 인가 봐',
 '금액 죠',
 '수강료 쯤 는가 요',
 '배달 주 시 나요',
 '뜨거운 이렇게 되 나요',
 '현금 결제 면 할인 되 나요',
 '커플 예약 신 맞 으신 가요',
 '차량 총 대 요',
 '단발 커트 파마 면 가격 정도 까요',
 '저번 살 별로 비싸 않 았었 는데요',
 '예약 방 들어갈 있',
 '쪽 맛있',
 '체크인 할 있 나요',
 '이것 할인 되 거 마찬가지 요',
 '요',
 '향수 있 나요',
 '체크인 시 면 되 나요',
 '카드 결제 되 나요',
 '개 한정 나왔',
 '쉽 게 발 넣 었 다가 뺄 있 그런 신발 원하 데 늘어나 죠',
 '사과 박스 포장 찢어졌 는데 바꿔 주 실 있 나요',
 '오늘 묵 방 있 나요',
 '후시 딘 마 데카 솔 게 더 좋 가요',
 '당구장 영업시간 시 에요',
 '일요일 쉬 시',
 '여자 옷 수선 세요',
 '넘어져서 바지 무릎 찢어졌 는데 수선 할 있',
 '요즘 추천 펌 뭔 가요',
 '주문 가능 한가요',
 '키즈 클럽 이랑 수영장 다 이용 할 있',
 '수학 금액 똑같 아요',
 '할인 카드 따로 없 나요',
 '시 문 닫 죠',
 '고기 반 근 정도 만 드릴까요',
 '헤나 염색 아니 죠',
 '히비스커스 라는 식물 이름 있',
 '주위 관광지 있 나요',
 '생일 쿠폰 쓰 카드 적립 되 나요',
 '아까 방 주',
 '호르몬제 먹 으면 빨리 좋 지나 요',
 '포장 이랑 가격 똑같 아요',
 '염색약 팔 아요',
 '카드 결제 가능 죠',
 '미리 주문 찾아가 거 되 고요',
 '쿠폰 있',
 '머리 러 왔 는데 기다려야 나요',
 '염색 면 머릿결 보다 상하 겠 죠',
 '사장 님 시 열

 '새로 나온 니트 어떠세요',
 '있',
 '소시지 볶음 있',
 '침대 트윈 인가요 싱글 인가요',
 '목걸이 금방 끊 진 않',
 '열',
 '분 이나 걸릴까요',
 '나이 액세서리 다니 면 잘 어울릴까 요',
 '내일 정오 가능 할까요',
 '원 쿠폰 뭐 예요',
 '순 살 주문 려고 는데 뼈 있 거 반반 씩 가능 할까요',
 '주중 날짜 예약 아직 가능 세요 정확 날짜 확인 주 시 겠',
 '커피 말 뭐 있 나요',
 '아이 크림 종류 있 나요',
 '머리 빠지 기 시작 했 는데 탈모 케어 있',
 '찾 으러 되 나요',
 '요즘 색상 거 많이 입',
 '박 예요',
 '이름 되 시 나요',
 '학교 예요',
 '애 피자 좋 하나 요 고등학생',
 '요 줄 문 제품 예요',
 '할인 쿠폰 사용 가능 죠',
 '차기 면 다 세워 요',
 '부대시설 스파 있 나요',
 '씨제이 카드 있 으면 적립 되 나요',
 '오늘 시 예약 했 는데요',
 '시 문 열',
 '전부 다 요',
 '차 세워 요',
 '번 신 볼 있',
 '삼겹살 돈 에요',
 '차 종류 게 있 나요',
 '신발 사이즈 되 세요',
 '꽃 더 추가 면 드나 요',
 '확장 되 걸로 번 볼 있',
 '그러면은 요 소고기 초밥 소고기 생 거 아니 죠',
 '가격 너무 비싸 서 거 추천 주 실 있 나요',
 '이거 카피 은데',
 '비슷 맛 커피 있 나요',
 '티 색상 하나 뿐 인가요',
 '분 세요',
 '실',
 '저거 뭐 요',
 '이벤트 뭐 있',
 '더치 커피 니 더치 라떼 차이점 있 나요',
 '블랙 색상 사이즈 없',
 '홈페이지 구입 할 있 나요',
 '까만색 옷 많이 나가 죠',
 '년 쯤 배우 면 대회 번 나가 나요',
 '바비큐 고기 류 거 준비 거 죠',
 '디자인 만 면 가능 다는 말씀 세요',
 '사이즈 없 으면 하나 요',
 '반찬 만 별도 구매 가능 한가요',
 '한우 양지 국거리 근 인가요',
 '주차장 없 나요',
 '받 으면 처진 얼굴 땡겨 그러 나요',
 '

In [204]:
pprint(y_train) #y에는 답이

array(['Q', 'Q', 'R', ..., 'Q', 'Q', 'Q'], dtype=object)


In [205]:
type(X_train)

list

In [206]:
len(X_train)

26183

In [207]:
from collections import Counter
#문장 단위로 가장 많이 나온 유형 확인
count = Counter(X_train)
x_train_morphs = dict(count.most_common())
print(x_train_morphs)

{'인가요': 61, '요': 52, '화장실 있': 44, '이거 예요': 32, '시': 30, '분 세요': 29, '배달 되 나요': 28, '계산 카드 가능 한가요': 25, '예요': 24, '자리 있 나요': 24, '분 신가요': 22, '시 문 열': 20, '가격 요': 20, '방 있 나요': 20, '옷 예요': 19, '가격 예요': 19, '카드 결제 가능 한가요': 19, '시 영업 나요': 18, '걸릴까요': 18, '죠': 18, '': 17, '가격 되 나요': 16, '카드 결제 되 나요': 16, '화장실 있 나요': 16, '걸려요': 16, '성함 되 시 죠': 15, '당 인가요': 15, '가격 인가요': 15, '게 잘 나가 요': 14, '뭐 맛있': 14, '와이파이 다 설치 되 있 나요': 14, '걸리 나요': 13, '배달 가능 한가요': 13, '이거 요': 13, '학년 에요': 13, '커플 석 있 나요': 13, '시 시 세요': 13, '색상 있 나요': 13, '사이즈 되 세요': 12, '비상구 쪽 있 나요': 12, '거 예요': 12, '배달 돼요': 12, '예약 가능 한가요': 11, '이건 예요': 11, '사이즈 있 나요': 11, '세요': 11, '성함 되 세요': 11, '화장실 요': 11, '성함 되 시 나요': 11, '쉬 날 있': 11, '가격': 11, '청소년 시 이용 가능 한가요': 10, '자리 있': 10, '포인트 적립 되 나요': 10, '음식 주문 가능 한가요': 10, '기다려야 나요': 10, '가방 예요': 9, '이거 가격 예요': 9, '와이파이 되 나요': 9, '현금영수증 되 나요': 9, '차 세워 요': 9, '예약 셨 나요': 9, '시 시': 9, '학교 인가요': 9, '일요일 세요': 9, '체크아웃 시 인가요': 9, '조끼 예요': 9, '방 있': 8, '이건 요': 8, '신가요': 8, '삼성 페이 결제 가능 한가요': 8

In [208]:
#형태소 단위로 가장 많이 나온 유형 확인해 보기.
mecab = Mecab()
word_tokens=[]
sentence = X_train
for word in sentence:
    word_tokens += mecab.morphs(word)
count_tokens = Counter(word_tokens)
count_words =dict(count_tokens.most_common())
print(count_words) #는 / 이 / 있 과 같은 조사들이 가장 많이 나옴
#나중에 tfdif 방식으로 vectorizer 하는것이 더 유리할 것!(bow 방식 보단.)

{'있': 5557, '나요': 5183, '요': 4977, '되': 2977, '시': 2410, '거': 2266, '죠': 1922, '면': 1766, '주': 1598, '나': 1311, '인가요': 1280, '가능': 1202, '없': 1072, '게': 1036, '예': 1019, '세요': 972, '는데': 967, '뭐': 948, '안': 919, '한가요': 819, '이거': 815, '할': 779, '만': 760, '다': 756, '가격': 725, '돼요': 649, '예약': 633, '가요': 619, '데': 618, '먹': 575, '할인': 561, '사이즈': 542, '원': 521, '겠': 514, '정도': 500, '예요': 495, '제품': 477, '잘': 476, '카드': 471, '하나': 467, '결제': 451, '에': 448, '실': 448, '신': 437, '옷': 435, '에요': 433, '님': 421, '주문': 419, '일': 416, '따로': 415, '건': 408, '분': 407, '많이': 401, '배달': 393, '더': 392, '좋': 391, '보': 384, '가방': 382, '방': 377, '해야': 360, '번': 358, '려고': 342, '입': 339, '이용': 335, '종류': 334, '아요': 330, '개': 321, '서': 316, '쯤': 309, '맞': 301, '들어가': 295, '싶': 287, '받': 281, '사': 280, '나가': 277, '찾': 275, '으면': 263, '인데': 261, '염색': 261, '은데': 260, '말': 259, '두': 256, '될까요': 253, '포장': 250, '으세요': 249, '랑': 244, '색상': 243, '해도': 241, '괜찮': 241, '수업': 240, '오늘': 238, '추천': 238, '추가': 237, '신

In [209]:
print(count_tokens.most_common(10)) #가장 많이 나온 상위 10개 토큰

[('있', 5557), ('나요', 5183), ('요', 4977), ('되', 2977), ('시', 2410), ('거', 2266), ('죠', 1922), ('면', 1766), ('주', 1598), ('나', 1311)]


In [210]:
print(count_tokens.most_common()[:-10:-1]) #가정 적게 나온 하위 10개 토큰

[('장미', 1), ('경옥고', 1), ('트랩', 1), ('줄여도', 1), ('바스켓', 1), ('덩이', 1), ('채워', 1), ('어반', 1), ('우엉조림', 1)]


In [211]:
count = Counter(y_train)
y_train_morphs = dict(count.most_common())
print(y_train_morphs) #문의 질문 dataset이 추천 질문 dataset 보다 약 20배 많다.

{'Q': 24884, 'R': 1299}


## TF-IDF 방식의 텍스트 데이터 벡터화 작업
- 형태소 단위로 문장을 쪼개 본 결과 전체 빈도수를 감안해 볼때 의미없는 -은/-는/-이 같은 조사를 빼고 '향인','페밀리룸으로'와 같이 1회정도만 등장하는 의미없는 빈도수가 낮은 형태소는 제외하는 것이 의미있음.
- 이떄 사용할 수 있는 방식이 TF-IDF 임.


In [212]:
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer

In [213]:
class CleansingData:
    
    # 정제된 text만 가져오기 위한 정규 표현식 적용_1
    def clean_str(self, text):
        
        pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
        text = re.sub(pattern=pattern, repl='', string=text)
        pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
        text = re.sub(pattern=pattern, repl='', string=text)
        pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
        text = re.sub(pattern=pattern, repl='', string=text)
        pattern = '<[^>]*>'         # HTML 태그 제거
        text = re.sub(pattern=pattern, repl='', string=text)
        pattern = '[^\w\s]'         # 특수기호제거
        text = re.sub(pattern=pattern, repl='', string=text)   
        return text 
    
    # 정제된 text만 가져오기 위한 정규 표현식 적용_2(이모지 제거)
    def no_imoji(self, text):

        emoji = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
        text= emoji.sub(r'', text)
        return text 

In [214]:
c = CleansingData()

In [215]:
print(X_train)

['빨리 나요 밥', '날개 봉 그건', '양 갱이 어른 좋', '상품권 신세계 상품권 만 되 나요', '앞 쓰여 있 던데 없', '애 이랑 따로 있 어야 조용 할 꺼 은데 분리 곳 조용 곳 있 나요', '빨리 게 김밥 죠', '아직 세트 할인 이벤트 있 나요', '요런 옷 예요', '이거 마음 드 는데 번 메 봐도 될까요', '요 카키색 코트 예요', '선행 하나 요', '앞 신용카드 사절 라고 되 있 데 체크카드 되 나요', '교재비 요', '색상 좋 으세요', '사장 님 수육 거리 뭐 있', '내과 서 피 검사 보 셨', '커플 석 거 있 나요', '이거 말 평소 잘 할 있 거 없', '할인 행사 한다고 었 는데 맞 나요', '교재비 매달 나요', '현재 흰색 검정 색 있 데 신 보 시 겠', '예요', '옷 예요', '맛있 사과 있 나요', '일 정도 걸리 나요', '색상 색상 밖에 없', '구 치 려고 는데 대 다이 있', '약밥 말 세요', '메뉴 있 거 만 있 라면 없', '유료 주차장 요', '세트 메뉴 면 가격 많이 싸 나요', '진로 면 고등학교 시작 경우 있 나요', '파마 걸리 나요', '요즘 어르신 생신 선물 이나 제일 잘 나가 케이크 거 에요', '클래식 곡 치 려면 걸릴까요', '주문 시 면 십 분 십 분 정도 걸리 세요', '시 영업 나요', '제품 어때요', '입 되 나요', '스타일 선호 아세요', '하루 숙박 비용 얼만 가요', '남녀 공용 인가 보', '회원 가격 인가요', '신어 볼 있 나요', '아플 마다 먹 거 예요', '남자 전용 미용실 맞 나요', '반코트 정도 할까요', '있 반찬 하나 씩 하나 요', '너무 요란 무늬 말 얌전 거 없', '선 게 잘 나가 요', '호텔 식당 일 층 아닌가요', '아이 결석 면 따로 연락 주 시 나요', '걸리 나요', '우유 있', '예약 가능 한가요', '샐러드 드레싱 있 나요', '딸 검 색 물들인 색상 빼 싶 요', '길 게 인절미 예요', '뚫 으시 겠', '사이즈 뭐 필

### TFIDF 예제

In [216]:
# sentence = [ "휴일인 오늘도 서쪽을 중심으로 폭염이 이어졌는데요, 내일은 반가운 비소식 이 있습니다."
#             , "폭염을 피해서 휴일에 놀러왔다가 갑작스런 비로 인해 망연자실 하고 있습니다."]

In [217]:
# # 객체 생성 
# tfidf_vectorizer = TfidfVectorizer() 
# # 문장 벡터화 진행 
# tfidf_matrix = tfidf_vectorizer.fit_transform(sentence) 
# # 각 단어 
# text = tfidf_vectorizer.get_feature_names()
# # 각 단어의 벡터 값 
# idf = tfidf_vectorizer.idf_

In [218]:
# print(dict(zip(text, idf)))

## TFIDF 함수화

In [219]:
# # def tfidf_features(X_train, X_test):
# #     #train 데이터 셋을 tf-idf로 벡터화 하려 함
# tfidf_vectorizer = TfidfVectorizer(min_df=5
#                                   ,max_df = 0.85
#                                   ,ngram_range=(1,3)
# #                                   ,token_pattern=c.clean_str
#                                   )

# X_train_vect = tfidf_vectorizer.fit_transform(X_train)
# X_test_vect = tfidf_vectorizer.fit_transform(X_test)

# return X_train, X_test, sorted(tfidf_vectorizer.vocabulary_.items())

In [220]:
# def tfidf_features(X_train, X_test):
#     #train 데이터 셋을 tf-idf로 벡터화 하려 함
tfidf_vectorizer = TfidfVectorizer(min_df = 5
                                  ,max_df = 0.85
                                  ,ngram_range=(1,3)
                                  ,token_pattern='(\S+)') #공백을 제거한 모든 문자/숫자/특수문자

tfidf_vectorizer.fit(X_train)
X_train_vect = tfidf_vectorizer.transform(X_train)

X_test_vect = tfidf_vectorizer.transform(X_test)

sorted(tfidf_vectorizer.vocabulary_.items()) #가나다 순으로 단어 사전 정렬

[('가게', 0),
 ('가게 앞', 1),
 ('가게 앞 주차', 2),
 ('가게 이름', 3),
 ('가격', 4),
 ('가격 가요', 5),
 ('가격 나요', 6),
 ('가격 다', 7),
 ('가격 달라요', 8),
 ('가격 달라지', 9),
 ('가격 대', 10),
 ('가격 대 정도', 11),
 ('가격 더', 12),
 ('가격 돼요', 13),
 ('가격 되', 14),
 ('가격 되 나요', 15),
 ('가격 되 죠', 16),
 ('가격 똑같', 17),
 ('가격 똑같 아요', 18),
 ('가격 만', 19),
 ('가격 비슷', 20),
 ('가격 세일', 21),
 ('가격 어떻', 22),
 ('가격 어떻 게', 23),
 ('가격 에요', 24),
 ('가격 예요', 25),
 ('가격 요', 26),
 ('가격 원', 27),
 ('가격 인가요', 28),
 ('가격 정도', 29),
 ('가격 죠', 30),
 ('가격 쯤', 31),
 ('가격 차이', 32),
 ('가격 차이 많이', 33),
 ('가격 차이 있', 34),
 ('가격 추가', 35),
 ('가격 추가 되', 36),
 ('가격 하나', 37),
 ('가격 하나 요', 38),
 ('가격 할인', 39),
 ('가격대', 40),
 ('가격대 되', 41),
 ('가격대 되 나요', 42),
 ('가격대 정도', 43),
 ('가까운', 44),
 ('가능', 45),
 ('가능 거', 46),
 ('가능 거 죠', 47),
 ('가능 나요', 48),
 ('가능 미용실', 49),
 ('가능 방', 50),
 ('가능 방 있', 51),
 ('가능 세요', 52),
 ('가능 죠', 53),
 ('가능 한가요', 54),
 ('가능 할까요', 55),
 ('가능 합니까', 56),
 ('가래떡', 57),
 ('가루', 58),
 ('가르쳐', 59),
 ('가르쳐 주', 60),
 ('가르치', 61),
 ('가르치 시', 62),
 ('

In [221]:
tfidf_vectorizer.vocabulary_ #{형태소 단어 : 백터화} 로 출력

{'빨리': 2736,
 '나요': 820,
 '밥': 2331,
 '봉': 2564,
 '그건': 633,
 '양': 3816,
 '어른': 3849,
 '좋': 5109,
 '상품권': 2927,
 '신세계': 3496,
 '만': 1713,
 '되': 1309,
 '만 되': 1729,
 '되 나요': 1325,
 '만 되 나요': 1730,
 '앞': 3772,
 '있': 4714,
 '던데': 1222,
 '없': 3886,
 '있 던데': 4752,
 '애': 3783,
 '이랑': 4537,
 '따로': 1481,
 '어야': 3856,
 '조용': 5081,
 '할': 6131,
 '꺼': 756,
 '은데': 4430,
 '분리': 2628,
 '곳': 486,
 '따로 있': 1499,
 '있 어야': 4769,
 '곳 있': 489,
 '있 나요': 4739,
 '곳 있 나요': 490,
 '게': 325,
 '김밥': 734,
 '죠': 5133,
 '아직': 3671,
 '세트': 3140,
 '할인': 6161,
 '이벤트': 4561,
 '할인 이벤트': 6182,
 '이벤트 있': 4565,
 '할인 이벤트 있': 6183,
 '이벤트 있 나요': 4566,
 '요런': 4265,
 '옷': 4179,
 '예요': 4094,
 '요런 옷': 4267,
 '옷 예요': 4189,
 '이거': 4470,
 '마음': 1701,
 '드': 1389,
 '는데': 952,
 '번': 2424,
 '메': 1946,
 '봐도': 2571,
 '될까요': 1363,
 '마음 드': 1702,
 '드 는데': 1391,
 '는데 번': 971,
 '메 봐도': 1949,
 '봐도 될까요': 2576,
 '마음 드 는데': 1703,
 '요': 4233,
 '카키색': 5605,
 '코트': 5684,
 '코트 예요': 5685,
 '선행': 3067,
 '하나': 6060,
 '하나 요': 6073,
 '라고': 1555,
 '데': 1227,

In [222]:
print(X_train_vect)

  (0, 2736)	0.6908136965968512
  (0, 2331)	0.6883774595311395
  (0, 820)	0.22116263202369457
  (1, 2564)	0.7071067811865475
  (1, 633)	0.7071067811865475
  (2, 5109)	0.432582658629653
  (2, 3849)	0.6529854850586368
  (2, 3816)	0.621676925545446
  (3, 3496)	0.42903756213860056
  (3, 2927)	0.6346783660146356
  (3, 1730)	0.4013165274896458
  (3, 1729)	0.3787935918260906
  (3, 1713)	0.208956965568195
  (3, 1325)	0.17713277693071672
  (3, 1309)	0.1458514439408865
  (3, 820)	0.11072002402457963
  (4, 4752)	0.5984629272925283
  (4, 4714)	0.2005856310551969
  (4, 3886)	0.3239522129144007
  (4, 3772)	0.48918586157310456
  (4, 1222)	0.5073063037805549
  (5, 6131)	0.13982545195002005
  (5, 5081)	0.5274884317909284
  (5, 4769)	0.2573822303952973
  (5, 4739)	0.1032641816894395
  :	:
  (26180, 6254)	0.4367201298100819
  (26180, 2574)	0.4315073474036095
  (26180, 2573)	0.41695761547949767
  (26180, 2571)	0.36323415082682775
  (26180, 1864)	0.4673972672329493
  (26180, 1325)	0.21856593580603986
  (261

In [223]:
print(X_test_vect)

  (0, 6228)	0.5314503397974962
  (0, 6226)	0.46588101696281087
  (0, 6221)	0.3495742635722459
  (0, 3140)	0.38713055655696216
  (0, 1325)	0.25609790230836915
  (0, 1309)	0.21087146878817126
  (0, 1028)	0.3045736570896734
  (0, 820)	0.1600785940781465
  (1, 2943)	0.44331179797157455
  (1, 2763)	0.3398106062614637
  (1, 1686)	0.4660607417713804
  (1, 1614)	0.522618883528994
  (1, 545)	0.44459002374079953
  (2, 6127)	0.3866239370694135
  (2, 4843)	0.48100642466707716
  (2, 4825)	0.27817524790647724
  (2, 3862)	0.4090723009722909
  (2, 1201)	0.4650790570056192
  (2, 1186)	0.29044081995249
  (2, 820)	0.13403489552111209
  (2, 325)	0.23624856055874915
  (3, 5484)	0.3131200739983617
  (3, 3390)	0.5114294164617204
  (3, 3318)	0.19041215727168448
  (3, 3245)	0.451536654882617
  :	:
  (8723, 558)	0.35767009735156824
  (8723, 42)	0.4315962909441356
  (8723, 41)	0.41984095881540195
  (8723, 40)	0.34252482831044395
  (8724, 5139)	0.2830607162347384
  (8724, 5134)	0.21526361555088913
  (8724, 4537)	

In [224]:
#X_train.shape, X_test.shape, y_train.shape, y_test.shape
#((26183,), (6403,), (26183,), (6403,))
X_train_vect.shape, X_test_vect.shape #특성컬럼 8541개로 같아야 함 => 
#이상했던 이유는 FIT 한 모델에 각각 다른 데이터를 집어 넣었기 때문.

((26183, 6403), (8728, 6403))

### 답변 column 부분 Vectorize 해주기

In [225]:
from sklearn.preprocessing import MultiLabelBinarizer

In [226]:
len(df[df['meaning']=='Q'])

33176

In [227]:
len(df[df['meaning']=='R'])

1735

In [228]:
mlb = MultiLabelBinarizer(classes=['Q','R'])

In [229]:
mlb

MultiLabelBinarizer(classes=['Q', 'R'], sparse_output=False)

In [230]:
y_train = mlb.fit_transform(y_train)

In [231]:
y_test = mlb.transform(y_test)

## Classifier(분류기) 만들기 / tf-idf 방식

In [232]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier

In [233]:
def train_classifier(X_train, y_train):
    return OneVsRestClassifier(LogisticRegression(penalty='l2'
                                                 , C=10
                                                 , max_iter = 1000)).fit(X_train, y_train)

In [234]:
X_train[:5]

['빨리 나요 밥', '날개 봉 그건', '양 갱이 어른 좋', '상품권 신세계 상품권 만 되 나요', '앞 쓰여 있 던데 없']

In [235]:
classifier_tfidf = train_classifier(X_train_vect, y_train)

In [236]:
y_test_predicted_labels_tfidf = classifier_tfidf.predict(X_test_vect)
y_test_predicted_scores_tfidf = classifier_tfidf.decision_function(X_test_vect)

In [237]:
y_test_predicted_labels_tfidf

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]])

In [238]:
y_test_predicted_scores_tfidf

array([[ 6.81254654, -6.81254654],
       [ 5.4490043 , -5.4490043 ],
       [ 1.5580949 , -1.5580949 ],
       ...,
       [ 6.9280133 , -6.9280133 ],
       [ 6.97008627, -6.97008627],
       [ 4.15674136, -4.15674136]])

In [239]:
#눈으로 결과를 확인하기엔 벡터화된 상태보다 
#기존 텍스트 방식이 편하므로 다시 텍스트 데이터롤 변환

y_test_inversed = mlb.inverse_transform(y_test)
y_test_inversed[:50]

[('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('R',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('R',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('R',)]

In [240]:
y_test_pred_inversed = mlb.inverse_transform(y_test_predicted_labels_tfidf)
y_test_pred_inversed[:100]

[('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('R',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('R',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('R',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',),
 ('Q',)]

In [241]:
print(X_test)

['다 세트 해야 되 나요', '레벨 마다 교재 새로 사', '게 한테 더 잘 어울리 나요', '스키 스케이트 강습 시 추가 수업료 내 나요', '있 다가 주문 시 겠', '손상 없 거 다 그죠 염색약 좋 은가 봐요', '수업 시 끝나 나요', '와인 색 계열 잘 어울리 실 거 은데 파마 안 시', '뷰 좋 방 따로 있 나요', '세면도구 구비 되 있 죠', '귀걸이 케이 케이 게 있', '수업 간 수강료 차이 있', '그렇게 오래 걸려요', '영업 시', '작심 룸 개월 만 원 시크릿 룸 개월 만 원', '이거 인가요', '원 나물 만 따로 담 아서 가져갈 없 나요', '주차 단속', '이렇게 쬐끄만 건 서 나와요', '일 주일 이틀 정도 올 있 나요', '아주 큰 빅 사이즈 제품 있 나요', '물 셀프 인가요', '몸 으슬으슬 추운데 빨리 낫 약 있', '신상 나온 거 예요', '호밀 식빵 없 나요', '오션 뷰 방 하나 주', '화이트 모카 거 예요', '단체 주문 할 있 나요', '청소년 시 이용 가능 한가요', '매장 식사 가능 한가 봐요', '일 주일 번', '운동 화 주문 싶 은데 해야 나요', '돈가스 판매 나요', '돈 코츠 라멘 돈가스 들어가 거 예요', '탈모 걱정 되 데 괜찮 시술 있', '색상 하나 예요', '실내 수영장 없 나요', '한방 영양 크림 가격', '현금영수증 주 세요', '옷 입 볼 있', '개월 해드릴까요', '그걸로 선물 포장 주 세요', '치즈 돈가스 예요', '삼월 십 일 쯤 모임 있 어서 안쪽 예약 려고 는데 면 돼요', '가방 손잡이 가죽 에요', '와이파이 다 설치 되 있 나요', '첫 방문 시 할인 라고 붙 있 데 아직 나요', '봉지 정도 인가요', '봉지 요', '평소 쓰 시 제품 있 으세요', '코고 소리 너무 커서 못하 겠 는데 바꿔 주 시 겠', '메뉴 게 잘 나가 요', '체크아웃 일 후 시 나와야 죠', '대패 씩 는데요', '먹 으면 칼칼 그런 건 없', '배달 시키 면 는데 쯤 걸려요', '가

In [242]:
animals = ['사자','호랑이','기린','사슴','원숭이']

In [243]:
print(','.join(animals))

사자,호랑이,기린,사슴,원숭이


In [244]:
# % 연산자로 하는 포맷팅 vs .format() => 걸리는 시간 비교시 %포멧팅이 더 짧음. 

In [245]:
# % 연산자 포멧팅 

In [246]:
for i in range(500):
    print('질문 내용:\t{}\n정답:\t{}\n예측:\t{}\n\n'.format(
        X_test[i],
        ','.join(y_test_inversed[i]),
        ','.join(y_test_pred_inversed[i])
    ))
    

질문 내용:	다 세트 해야 되 나요
정답:	Q
예측:	Q


질문 내용:	레벨 마다 교재 새로 사
정답:	Q
예측:	Q


질문 내용:	게 한테 더 잘 어울리 나요
정답:	Q
예측:	Q


질문 내용:	스키 스케이트 강습 시 추가 수업료 내 나요
정답:	Q
예측:	Q


질문 내용:	있 다가 주문 시 겠
정답:	Q
예측:	Q


질문 내용:	손상 없 거 다 그죠 염색약 좋 은가 봐요
정답:	Q
예측:	Q


질문 내용:	수업 시 끝나 나요
정답:	Q
예측:	Q


질문 내용:	와인 색 계열 잘 어울리 실 거 은데 파마 안 시
정답:	Q
예측:	Q


질문 내용:	뷰 좋 방 따로 있 나요
정답:	Q
예측:	Q


질문 내용:	세면도구 구비 되 있 죠
정답:	Q
예측:	Q


질문 내용:	귀걸이 케이 케이 게 있
정답:	Q
예측:	Q


질문 내용:	수업 간 수강료 차이 있
정답:	Q
예측:	Q


질문 내용:	그렇게 오래 걸려요
정답:	Q
예측:	Q


질문 내용:	영업 시
정답:	Q
예측:	Q


질문 내용:	작심 룸 개월 만 원 시크릿 룸 개월 만 원
정답:	Q
예측:	Q


질문 내용:	이거 인가요
정답:	Q
예측:	Q


질문 내용:	원 나물 만 따로 담 아서 가져갈 없 나요
정답:	Q
예측:	Q


질문 내용:	주차 단속
정답:	Q
예측:	Q


질문 내용:	이렇게 쬐끄만 건 서 나와요
정답:	Q
예측:	Q


질문 내용:	일 주일 이틀 정도 올 있 나요
정답:	Q
예측:	Q


질문 내용:	아주 큰 빅 사이즈 제품 있 나요
정답:	Q
예측:	Q


질문 내용:	물 셀프 인가요
정답:	Q
예측:	Q


질문 내용:	몸 으슬으슬 추운데 빨리 낫 약 있
정답:	R
예측:	Q


질문 내용:	신상 나온 거 예요
정답:	Q
예측:	Q


질문 내용:	호밀 식빵 없 나요
정답:	Q
예측:	Q


질문 내용:	오션 뷰 방 하나 주
정답:	Q
예측:	Q


질문 내용:	화이트 모카 거 예요
정답:	Q
예측:	Q


질문 내용:	단체 주문 할 있 나요
정답:	Q
예측

In [247]:
#양갱이 어른에게 좋을까요? 라는 질문에 예측은 Q, 정답은 R로 됨. 성능이 부족함!

## 예측-정답 확률을 보다 명확하기 확인하기 위해 평가함

In [248]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

In [249]:
def evaluation_scores(y_test, predicted):
    print('accuracy_score:', accuracy_score(y_test, predicted))
    
    f1_macro = f1_score(y_test, predicted, average='macro')
    f1_micro = f1_score(y_test, predicted, average='micro')
    f1_weighted = f1_score(y_test, predicted, average='weighted')
    
    print('F1-SCORE MACRO:', f1_macro)
    print('F1-SCORE MICRO:', f1_micro)
    print('F1-SCORE WEIGHTED:', f1_weighted)
    
    print('Precision macro:', average_precision_score(y_test, predicted, average='macro'))
    print('Precision micro:', average_precision_score(y_test, predicted, average='micro'))
    print('Precision weighted:', average_precision_score(y_test, predicted, average='weighted'))

In [250]:
y_test.shape, y_test_predicted_labels_tfidf.shape

((8728, 2), (8728, 2))

In [251]:
print('tfidf')
evaluation_scores(y_test, y_test_predicted_labels_tfidf)

tfidf
accuracy_score: 0.9642529789184234
F1-SCORE MACRO: 0.7684665685143184
F1-SCORE MICRO: 0.9642529789184234
F1-SCORE WEIGHTED: 0.9601059953937913
Precision macro: 0.663408853572685
Precision micro: 0.9476573178938419
Precision weighted: 0.9405713887027873


In [252]:
df = pd.read_csv('./question_tag.csv', index_col=0, encoding='utf-8-sig')
df.shape

(34911, 3)

In [283]:
df[150:160]

,sentence,main,meaning
150,닭고기는 국내산 쓰나요?,식재료문의,문의
151,주문하면 시간은 얼마나 걸려요?,배달시간문의,문의
152,네 수고하십니다 저 죄송하지만 배달 몇 시부터 몇 시까지 하나요?,영업시간문의,문의
153,어떤 메뉴들이 있나요?,메뉴문의,문의
154,가격이 얼마에요 몇 인분이고?,가격문의,문의
155,한 세트에 19000원요 그 종류 말고 다른 거 또 뭐 잘 나가는 거 뭐 있어요?,베스트메뉴문의추천요청,추천
156,후라이드는 얼마에요?,가격문의,문의
157,배달비 따로 드려야 되나요?,배달비문의,문의
158,배달 시키면 오는데 시간이 얼마쯤 걸려요?,배달시간문의,문의
159,어디신데요,배달시간문의,문의


In [254]:
korean_re = re.compile('[^ ㄱ-ㅣ가-힣]+')

In [255]:
def extract_last_word(text):
    #한글과 공백 제외한 다른 문자는 삭제
    text = korean_re.sub('',text)
    
    #공백을 기준으로 text를 split 한 뒤 맨 마지막 단어만 저장
    text = text.split()[-1]
    
    return text

In [256]:
last_words=df.sentence.apply(extract_last_word).tolist()
len(last_words)

34911

In [257]:
last_words = list(set(last_words))
len(last_words)

5735

In [258]:
pprint(last_words[:20])

['넓네요',
 '봄신상인가봐요',
 '돌려주시죠',
 '체육고입니까',
 '무료인가요',
 '이건무슨떡이에요',
 '같으세요',
 '포장해주시겠어요',
 '다녀요',
 '들어오죠',
 '여셔요',
 '하셨어요',
 '곳은',
 '계열이죠',
 '과정인가요',
 '센티죠',
 '할인될까요',
 '기준이에요',
 '언제까지에요',
 '양배추']


- 기존 질문셋 데이터서 가져온 질문문장 중 어미만 뽑아 본 결과 중복제외 5735개
- 이 중 유의미하고 말이 되는(질문다운) 질문을 만들기 위해 질문 어미를 따로 설정
- 질문 말뭉치를 따로 생성해 보고자 함

In [259]:
question_words=['가르쳐주세요','궁금하네요','궁금한데요','궁금합니다','궁금해서요','궁금해요'
                ,'물어볼게요','물어볼게예','물어볼라는데요','물어볼려고요','물어봐야겠죠'
                ,'뭐가있습니까','뭐드라','뭐뭐있어요','뭐에요','뭐였어요','뭐예요','뭐있어요'
               ,'뭐죠','뭐지','뭐지요','뭐가있죠','뭔가요','뭐고','뭔고','뭔데예','뭔데요','뭘까요','뭡니까'
               ,'부탁해','부탁드려요','아세요','아시나요','알고계신가요','알려주나요','알려주세요','알려주시겠습니까'
               ,'알려주시겠어요','알려주실래요','알려줘요','알려줘여','알려주삼','알려줄래','알려줄수있나요'
                ,'알수있나요','알아보는데요','알아보려고요','알아','알아요','어디에요'
               ,'어디에있어요','어디였죠','어디에요','어디예요','어디인가요','어디입니까'
               ,'어디인데요','어디있나요','어디있어요','어디있죠','어디죠','여쭈어볼려고요'
               ,'여쭤보려고요','여쭤보려구요','여쭤보려하는데요','여쭤볼라고','여쭤볼려고'
               ,'여쭤볼려구요','없어요','없습니까','있나요','있겠죠','있나','있나보네요','있나봐예'
               ,'있나요','있노','있는가','있는가봐요','있는가예','있는가요','있는갑네요','있는거죠'
               ,'있는건가요','있는지','있는지요','있는지궁금해','있는지궁금해요','있다면서요','있습니까'
               ,'있어요','있어','있나','있어여','있어예','있었나요','있으려나','있으세요','있으시죠'
               ,'있으신가요','있으신거죠','있으신지요','있을가요','있을까요','있죠','있지','있지요'
               ,'질문합니다','질문드려요','찾아줘','찾아주세요','추천고고','추천좀해줘','추천좀'
               ,'추천해','추천해주겠니','추천해줘','추천해줘라','추천해라','추천해봐','추천해줘봐','추천해줄수있어'
                ,'추천부탁드려요','추천하세요','추천해주세요','추천해주시겠어요','추천해주실래요','추천부탁해'
               ,'추천부탁해요','추천가능해','추천해줄래','추천해봐','추천안해줄꺼니','추천해줄꺼니','추천해봐']

In [260]:
list(set(question_words))

['어디예요',
 '어디입니까',
 '어디인가요',
 '있다면서요',
 '어디에요',
 '뭔가요',
 '찾아주세요',
 '있을가요',
 '있는가',
 '있으려나',
 '부탁해',
 '알아보려고요',
 '물어볼려고요',
 '찾아줘',
 '있는거죠',
 '여쭤볼려구요',
 '가르쳐주세요',
 '추천해줄래',
 '추천해주실래요',
 '알려주시겠습니까',
 '추천해봐',
 '아시나요',
 '물어볼게요',
 '있으신가요',
 '뭘까요',
 '알아보는데요',
 '뭐죠',
 '어디있어요',
 '여쭤보려구요',
 '여쭤볼라고',
 '아세요',
 '추천해줄꺼니',
 '있으신거죠',
 '있나보네요',
 '있는가예',
 '궁금해요',
 '여쭤보려고요',
 '알려주세요',
 '뭡니까',
 '추천해줘',
 '추천해주세요',
 '추천좀',
 '없습니까',
 '추천좀해줘',
 '있는지궁금해',
 '궁금한데요',
 '뭐가있습니까',
 '뭔고',
 '질문합니다',
 '물어볼라는데요',
 '어디죠',
 '물어봐야겠죠',
 '있죠',
 '있는갑네요',
 '있습니까',
 '뭐예요',
 '알려주시겠어요',
 '추천부탁해요',
 '추천해주시겠어요',
 '있는지',
 '있어요',
 '있으세요',
 '어디인데요',
 '뭐고',
 '있어',
 '있는건가요',
 '추천해줘라',
 '추천가능해',
 '알려줘요',
 '뭔데예',
 '알려줘여',
 '없어요',
 '뭔데요',
 '있으신지요',
 '있노',
 '추천안해줄꺼니',
 '추천해라',
 '알려줄래',
 '어디에있어요',
 '있나',
 '어디있죠',
 '있는가봐요',
 '알아요',
 '뭐가있죠',
 '알려주실래요',
 '알려줄수있나요',
 '있나봐예',
 '있지',
 '알고계신가요',
 '있는가요',
 '추천부탁드려요',
 '추천해주겠니',
 '궁금해서요',
 '추천해줄수있어',
 '어디였죠',
 '궁금하네요',
 '있나요',
 '추천해',
 '물어볼게예',
 '궁금합니다',
 '있을까요',
 '부탁드려요',
 '있어예',
 '있었나요',
 '뭐드라',
 

In [261]:
question_words = list(set(question_words))

In [262]:
len(question_words)

130

## 문의 / 추천 카테고리에 맞는 질문 생성

- 질문 데이터 셋을 크게 문의와 추천, 2가지로 나누어서 진행하였다.
- 이 가운데 문의는 1)장소 관련 문의와 2)리뷰 관련 문의 3)가격 관련 문의로 나누고
- ex) 준오헤어 강남2호점 어디에 있어요? / 준오헤어 강남 2호점 커트 잘해요? / 준오헤어 강남2호점 남자 커트 얼마 정도 하나요?

- 추천은 총 6가지 항목을 기반으로 진행, 1) 청결 / 2) 서비스(고객 만족도) / 3) 매장 분위기,인테리어 / 4) 가격 / 5) 위치/교통 , 6)헤어 관련 만족도(디자이너 실력, 시술 후 헤어 상태 등 헤어 관련 키워드) 로 해당 항목 기준을 기반하여 추천
- ex) 강남역에서 댄디컷 가장 잘하는곳 어디에요? / 강남역에서 실내 넓고 깨끗한 미용실 어디에 있어요? / 디자이너 친절하고 꼼꼼하게 커트해 주는 미용실 강남역에 있어요? 등등

- 헤어스타일 관련 이미지 분석을 통한 질문은 따로 생성하지 않음

In [263]:
#위 기준에 맞는 질문 어미 만들기는 구글 드라이브에 별도로 만들어 저장함.

In [264]:
question_tail = pd.read_excel('./문의 _추천 질문 말뭉치 만들기.xlsx'
                            ,encoding='utf-8-sig'
                           )
question_tail.head()

,Req_question_1,Req_question_2,Req_sol_question,Que_loc_1,Que_loc_2,Que_sol_loc,Que_review_1,Que_review_2,Que_sol_review,Que_price_1,Que_price_2,Que_sol_price
0,아는 곳,가르쳐주세요,가르쳐주세요,주소,어디에요,어디에요,사람들 리뷰,가르쳐주세요,어때?,가격,어때?,어때?
1,아는 데,궁금하네요,궁금하네요,상세주소,어디에있어요,어디에있어요,사람들 반응,궁금하네요,어때요?,비용,어때요?,어때요?
2,괜찮은 곳,궁금한데요,궁금한데요,자세한 주소,어디였죠,어디였죠,리뷰,궁금한데요,어떠니?,값,어떠니?,어떠니?
3,괜찮은 데,궁금합니다,궁금합니다,위치,어디에요,어디에요,반응,궁금합니다,어떤가요?,요금,어떤가요?,어떤가요?
4,쩌는 곳,궁금해서요,궁금해서요,자세한 위치,어디예요,어디예요,평,궁금해서요,어떻죠?,액수,어떻죠?,어떻죠?


In [265]:
#컬럼에 들어간 데이터의 길이(갯수)가 각각 다르므로 별도로 처리, 다시 저장해 주자
req_question_1 = question_tail.Req_question_1.dropna().tolist()
req_question_2 = question_tail.Req_question_2.dropna().tolist()
req_sol_question = question_tail.Req_sol_question.dropna().tolist()

In [266]:
que_question_1 = question_tail.Que_loc_1.dropna().tolist()
que_question_2 = question_tail.Que_loc_2.dropna().tolist()
que_sol_question = question_tail.Que_sol_loc.dropna().tolist()

In [267]:
que_review_1 = question_tail.Que_review_1.dropna().tolist()
que_review_2 = question_tail.Que_review_2.dropna().tolist()
que_sol_review = question_tail.Que_sol_review.dropna().tolist()

In [268]:
que_price_1 = question_tail.Que_price_1.dropna().tolist()
que_price_2 = question_tail.Que_price_2.dropna().tolist()
que_sol_price = question_tail.Que_sol_price.dropna().tolist()

In [269]:
#추천 질문 어미 생성을 위한 빈 리스트 만들기
req_list=[]

#req_question_1, req_question_2 를 조합한 추천 질문 만들기
for r1 in req_question_1:
    for r2 in req_question_2:
        text = r1 + ' ' + r2
        req_list.append(text)
#req_sol_question으로 단독 추천 질문 만들기
for r in req_sol_question:
    req_list.append(r)

In [270]:
#위와 같은 방식으로 장소 문의 질문 만들기
#장소 질문 어미 생성을 위한 빈 리스트 만들기
que_loc_list=[]

#req_question_1, req_question_2 를 조합한 추천 질문 만들기
for q1 in que_question_1:
    for q2 in que_question_2:
        text = q1 + ' ' + q2
        que_loc_list.append(text)
#que_sol_question으로 단독 추천 질문 만들기
for q in que_sol_question:
    que_loc_list.append(q)

In [271]:
#위와 같은 방식으로 장소 문의 질문 만들기
#가격 질문 어미 생성을 위한 빈 리스트 만들기
que_review_list=[]

#req_question_1, req_question_2 를 조합한 추천 질문 만들기
for re1 in que_review_1:
    for re2 in que_review_2:
        text = re1 + ' ' + re2
        que_review_list.append(text)
#que_sol_review으로 단독 추천 질문 만들기
for re in que_sol_review:
    que_review_list.append(re)

In [272]:
#위와 같은 방식으로 장소 문의 질문 만들기
#가격 질문 어미 생성을 위한 빈 리스트 만들기
que_price_list=[]

#req_question_1, req_question_2 를 조합한 추천 질문 만들기
for p1 in que_price_1:
    for p2 in que_price_2:
        text = p1 + ' ' + p2
        que_price_list.append(text)
#que_sol_price으로 단독 추천 질문 만들기
for p in que_sol_price:
    que_price_list.append(p)

In [273]:
#질문 조합 중 중복된 질문은 삭제하도록 한다
req_list = list(set(req_list))
que_loc_list = list(set(que_loc_list))
que_review_list= list(set(que_review_list))
que_price_list = list(set(que_price_list))

In [274]:
len(req_list), len(que_loc_list), len(que_review_list), len(que_price_list)

(12875, 459, 3573, 495)

In [275]:
print(req_list[50:100])

['실패한적 없는 곳 알수있나요', '괜찮은 헤어샵 어데로가는교', '훌륭한 데 뭐죠', '잘해주는 헤어샵 알려줄수 있나요', '추천할 곳 뭐였어요', '괜찮은 헤어샵 뭐에요', '실패한적 없는 미용실 있는가', '데려오고 싶은 헤어샵 질문드려요', '오지는 미용실 궁금한데요', '실패한적 없는 곳 궁금하네요', '잘해주는 헤어샵 어디에요', '요즘 잘나가는 곳 물어봐야겠죠', '지리는 데 뭐지', '추천가능한 곳 어딘지아나', '최고인 데 있나요', '걱정없는 데 있는지', '알려주고 싶은 헤어샵 있는가봐요', '요즘 잘나가는 데 뭐지', '추천가능한 헤어샵 있다면서요', '데려오고 싶은 데 있으시죠', '알려주고 싶은 곳 어디인지 알까요', '쩌는 미용실 어디에있죠', '같이오고 싶은 미용실 궁금해서요', '좋은 미용실 여쭤보려고요', '지리는 헤어샵 뭐가 있습니까', '실패한적 없는 데 뭐뭐있어요', '잘해주는 미용실 알려주시겠어요', '걱정없는 곳 알려주실래요', '실패한적 없는 데 어디였죠', '실패한적 없는 데 질문드려요', '걱정없는 헤어샵 있으신지요', '지리는 데 있나보네요', '예쁘게 하는 곳 여쭤볼려구요', '아는 헤어샵 있다면서요', '잘하는 헤어샵 여쭤볼라고', '대박 미용실 어뎁니까', '예쁘게 하는 헤어샵 있는거죠', '훌륭한 곳 있겠죠', '예쁘게 하는 미용실 어디에있죠', '데려오고 싶은 헤어샵 있는지요', '같이오고 싶은 미용실 있다면서요', '지리는 곳 어딘지아나', '같이오고 싶은 데 뭘까요', '걱정없는 데 어디죠', '실패하지않는 헤어샵 있으려나', '요즘 잘나가는 곳 부탁드려요', '실패하지않는 미용실 가르쳐주세요', '실패한적 없는 미용실 어디에있어요', '실패한적 없는 미용실 뭘까요', '추천가능한 곳 알려줘요']


In [276]:
print(que_loc_list[50:100])

['장소 어데인지아나', '말해줘', '자세한 위치 어디에있어요', '어디있죠', '주소 어디야', '주소 어디 위치해있죠', '자세한 주소 물어봐도 될까', '어디인지 알려줘요', '어뎁니까', '자세한 장소 어딥니까', '장소 어디에요', '장소 어데로가는교', '주소 알려줄수 있어', '자세한 장소 어디에있습니까', '상세주소 어뎁니까', '상세주소 어데로가나', '상세한 장소 아냐 모르냐', '자세한 위치 말해줘요', '자세한 위치 아냐', '자세한 장소 아냐', '상세주소 알려줄수 있어', '장소 알려줘', '자세한 위치 말해줘', '자세한 장소 아냐 모르냐', '주소 어딥니까', '자세한 장소 물어봐요', '상세한 장소 어디예요', '상세주소 아냐 모르냐', '장소 어디예요', '상세주소 어디야', '어딘지아나', '자세한 주소 어데로가는교', '자세한 주소 어디에있나요', '상세한 장소 어디있어요', '위치 어디인지 알려줘요', '자세한 주소 어디일까', '위치 어디', '어데로가나', '어디인지', '어디예요', '상세주소 어디에있나요', '자세한 위치 어디야', '상세주소 찾아줘요', '위치 어딘지아나', '장소 어디있나요', '자세한 위치 어딥니까', '자세한 주소 알까', '주소 알려줘요', '상세주소 물어봐요', '상세한 장소 어데로가는교']


In [277]:
print(que_review_list[50:100])

['평 알아보려고요', '리뷰 있지', '거짓없는 평점 있는가예', '리뷰 알아보는데요', '평가 뭡니까', '솔직한 평가 궁금해서요', '솔직한 평가 알려줄수있나요', '가본사람들 리뷰 여쭤볼려고', '사람들 평가 있어예', '거짓없는 평점 뭐뭐있어요', '진짜후기 알아', '평가 있나', '리뷰 뭐였어요', '후기 물어볼려고요', '거짓없는 리뷰 뭐드라', '사람들 리뷰 있었나요', '평 있으신가요', '솔직평가 있으신가요', '진짜평가 있나보네요', '사람들 반응 여쭤볼려고', '거짓없는 후기 있습니까', '진짜반응 있으시죠', '가본사람들 반응 있지', '가본사람들 평 알수있나요', '거짓없는 평 뭐가있죠', '진짜리뷰 알수있나요', '가본사람들 반응 있으신지요', '실제평가 아시나요', '거짓없는 평가 알려주나요', '진짜반응 알려주시겠어요', '거짓없는 후기 있는지', '사람들 평가 뭐지요', '가본사람들 후기 있나', '진짜리뷰 있으시죠', '사람들 후기 있다면서요', '반응 알려줄수있나요', '거짓없는 평 있죠', '거짓없는 평 질문합니다', '거짓없는 후기 물어봐야겠죠', '가본사람들 평가 알려주시겠어요', '가본사람들 평 있다면서요', '거짓없는 평점 궁금해요', '실제후기 뭐지요', '거짓없는 리뷰 아세요', '가본사람들 평가 알려주삼', '평 있을가요', '사람들 리뷰 부탁드려요', '가본사람들 리뷰 뭐였어요', '솔직반응 알아보는데요', '거짓없는 평가 뭐죠']


In [278]:
print(que_price_list[50:100])

['액수 찾아줘', '비용 어때?', '액수 알아', '하는 요금 어떻지?', '액수 알아보려고요', '하는 비용 질문드려요', '요금 어떻지?', '하는 비용 어떻지?', '값 질문합니다', '비용 어떨까?', '하는 요금 알아보려고요', '비용 알아', '요금 찾아줄래', '액수 여쭈어볼려고요', '물어볼라는데요', '하는 가격 알려줄수있나요', '하는 요금 물어봐야겠죠', '알려줄수 있나요', '얼마죠?', '하는 요금 질문합니다', '하는 비용 궁금하네요', '요금 알아보는데요', '요금 얼마입니까?', '액수 부탁드려요', '액수 어떻지?', '요금 궁금해서요', '가격 알려줄래', '하는 비용 여쭤보려구요', '값 부탁드려요', '비용 찾아줄래', '가격 알려주세요', '값 알고계신가요', '액수 여쭤보려하는데요', '가격 찾아주세요', '알아요', '하는 가격 알아요', '비용 물어봐야겠죠', '알려주실래요', '하는 요금 어때?', '가격 궁금해요', '요금 아세요', '비용 알아보는데요', '하는 비용 알려줘여', '액수 여쭤보려구요', '하는 비용 알아보는데요', '하는 비용 어떤가요?', '액수 알아보는데요', '물어볼게예', '비용 찾아주세요', '하는 가격 여쭤볼려구요']


In [279]:
#각각의 질문 리스트에서 인덱스를 key로, 질문 어미 조각을 value로 하여 
#딕셔너리에 저장
#len(req_list), len(que_loc_list), len(que_review_list), len(que_price_list)
req_list_dict = {}
que_loc_list_dict ={}
que_review_list_dict = {}
que_price_list_dict = {}

for i,req in enumerate(req_list):
    req_list_dict[i] = req

for i,loc in enumerate(que_loc_list):
    que_loc_list_dict[i] = loc
    
for i,review in enumerate(que_review_list):
    que_review_list_dict[i] = review
        
for i,price in enumerate(que_price_list):
    que_price_list_dict[i] = price

In [280]:
#위 딕셔너리를 pickle로 저장
import pickle

In [281]:
with open('req_list_dict.pickle', 'wb') as f:
    pickle.dump(req_list_dict, f, pickle.HIGHEST_PROTOCOL)
with open('que_loc_list_dict.pickle', 'wb') as f:
    pickle.dump(que_loc_list_dict, f, pickle.HIGHEST_PROTOCOL)
with open('que_review_list_dict.pickle', 'wb') as f:
    pickle.dump(que_review_list_dict, f, pickle.HIGHEST_PROTOCOL)
with open('que_price_list_dict.pickle', 'wb') as f:
    pickle.dump(que_price_list_dict, f, pickle.HIGHEST_PROTOCOL)

In [282]:
#피클로 저장해 둠으로서 질문 object를 그대로 저장, 
#load 함수를 통해 저장했었던 객체를 그대로 불러와 사용 가능